In [3]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.1592 ,random_state=0 , l1_ratio= 0.9498, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 48.91 , C = 553.73 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_15_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [8]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=862314, Mon Apr 13 13:31:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.463031735541200e+03 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
    2     26 6.404569639063930e+03 1.1e+00 1.20e-01  1e-01  1e-01 0:00.0
    3     39 6.368956440681479e+03 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6477.85064601] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6466.37177261] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -1.227735667007193e+03 1.8e+00 6.28e-03  5e-03  6e-03 0:00.9
  200   2600 -1.237382821659338e+03 1.9e+00 1.03e-04  7e-05  9e-05 0:01.7
  300   3900 -1.237384459528732e+03 2.1e+00 1.59e-06  9e-07  1e-06 0:02.5
  400   5200 -1.237384459830643e+03 2.3e+00 1.32e-08  6e-09  8e-09 0:03.4
  483   6279 -1.237384459830656e+03 2.4e+00 2.45e-09  1e-09  1e-09 0:04.1
termination on tolfun=1e-11 (Mon Apr 13 13:31:50 2020)
final/bestever f-value = -1.237384e+03 -1.237384e+03
incumbent solution: [-0.22138125  0.19791505  0.00072196  0.04862321 -0.1407633  -0.08111686
  0.05019263  0.05339507 ...]
std deviations: [1.31931105e-09 1.07509143e-09 1.13121328e-09 1.25507453e-09
 1.13585683e-09 1.38009115e-09 1.20947073e-09 1.35479352e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=888940, Mon Apr 13 13:31:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.995327654266527e+03 1.0e+00 1.19e-01  1e-01  1e-01 0:00.0
termination on tolf

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5995.32765427] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5995.327654, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5995.327654, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

   63    819 -3.214756254386721e+09 1.7e+00 1.26e+02  1e+02  1e+02 0:00.7
termination on tolfacupx=1000.0 (Mon Apr 13 13:31:51 2020)
final/bestever f-value = -3.196443e+09 -3.214756e+09
incumbent solution: [-79.67690741 268.03725605 244.58319387 336.4910751  -61.291907
 -78.19498904 115.4500699  -54.20950775 ...]
std deviations: [119.12389293 117.86344655 111.63950229 113.3423847  121.01978721
 121.88928379 122.66515099 119.08368468 ...]
Run : 1
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=894013, Mon Apr 13 13:31:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.484753309903610e+03 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 6.484328310755527e+03 1.1e+00 1.25e-01  1e-01  1e-01 0:00.0
    3     39 6.482229666508705e+03 1.1e+00 1.28e-01  1e-01  1e-01 0:00.0


...

   72    936 -3.836376572991874e+09 1.8e+00 1.40e+02  1e+02  1e+02 0:00.8
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:52 2020)
final/bestever f-value = -3.554820e+09 -3.836377e+09
incumbent solution: [ -96.77055212  -68.83259754   14.18602379   49.15044719  -63.74299996
 -260.91887082  117.21562835  -95.27031771 ...]
std deviations: [127.18709987 127.36106798 130.89474314 129.61538782 136.82053662
 134.41848844 133.30957844 136.4582572  ...]
Run : 29
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=873918, Mon Apr 13 13:34:52 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 6.484738200819794e+03 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 6.483535214915756e+03 1.1e+00 1.18e-01  1e-01  1e-01 0:00.0
    3     39 6.476590636283755e+03 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6484.98552073] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6484.83623508] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -1.223635785524318e+03 1.8e+00 8.79e-03  7e-03  8e-03 0:00.8
  200   2600 -1.237381205227065e+03 2.1e+00 1.37e-04  9e-05  1e-04 0:01.7
  300   3900 -1.237384458971615e+03 2.3e+00 2.25e-06  1e-06  2e-06 0:02.5
  400   5200 -1.237384459830510e+03 2.4e+00 4.62e-08  2e-08  3e-08 0:03.4
  500   6500 -1.237384459830659e+03 2.8e+00 2.18e-09  9e-10  1e-09 0:04.2
  501   6513 -1.237384459830659e+03 2.8e+00 2.16e-09  9e-10  1e-09 0:04.2
termination on tolfun=1e-11 (Mon Apr 13 13:34:57 2020)
final/bestever f-value = -1.237384e+03 -1.237384e+03
incumbent solution: [-0.22138125  0.19791506  0.00072195  0.04862321 -0.1407633  -0.08111686
  0.05019262  0.05339507 ...]
std deviations: [1.18888870e-09 1.05910698e-09 9.41077469e-10 1.13927396e-09
 9.34571495e-10 1.17258862e-09 1.24325842e-09 1.18560992e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=802286, Mon Apr 13 13:34:58 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.995

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5995.32765427] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5995.327654, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5995.327654, sigma=1.19e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

   66    858 -3.524333832682150e+09 2.1e+00 1.19e+02  1e+02  1e+02 0:00.7
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:58 2020)
final/bestever f-value = -3.152591e+09 -3.524334e+09
incumbent solution: [ 153.36223292 -276.30826251  254.89778575   85.74491979 -279.48287415
   76.96030157  235.62703656  386.99524302 ...]
std deviations: [115.80421394 110.40397717 101.51428852 108.45075798 117.49448395
 108.33854705 113.66609418 124.32531851 ...]


In [9]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F15_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:25])
    SVM_Fun [i] = F15(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F15(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([826.87055485]), count=array([1])) 1.9299256756318637e-06
SVM
ModeResult(mode=array([749.10384314]), count=array([1])) 55.838675038061396
ELN
ModeResult(mode=array([7.1756798e+11]), count=array([1])) 1.2546417222965653e+33
